In [1]:
import numpy as np


In [29]:
np.random.seed(1234567890)


In [30]:
data_properties = { 'n_obs': 10000,
                    'beta': np.array([0.25, 0.75, -2.25]),
                    'sigma': 5
                    }

In [31]:
def gen_data(data_properties):
    """
        WRITE DOCS
    """
    import numpy
    
    n_obs = data_properties['n_obs']
    beta  = data_properties['beta']
    n_x = data_properties['beta'].shape[0] - 1
    
    # make explanatory variables
    iota     = numpy.ones(n_obs).reshape(n_obs, 1)
    expl_var = numpy.random.uniform(size=[n_obs, n_x])
    
    X = numpy.column_stack(tup = (iota, expl_var))

    epsilon = numpy.random.normal(loc = 0, scale = data_properties['sigma'], size = n_obs)

    y = X @ beta + epsilon

    data = np.column_stack([y, X[:,1:]])

    return data #[y, X]


In [32]:
my_data = gen_data(data_properties)

In [38]:
my_data[:,0]

array([  0.71854923, -13.34252351,  -3.7721326 , ...,  11.59416204,
         2.83101472,  -1.30604114])

In [56]:
def ols_coeffficients(y, X, constant = True):

    assert type(y) == np.ndarray
    assert type(X) == np.ndarray
    assert type(constant) == bool

    assert y.shape[0] == X.shape[0]

    n_obs = y.shape[0]
    if constant == True:
        # add a constant to matrix

        iota     = np.ones(n_obs).reshape(n_obs, 1)
        X = np.column_stack([iota, X])
    
    # compute beta_hat
    XX = X.T.dot(X)
    Xy = X.T.dot(y)
    XX_inv = np.linalg.inv(XX)

    beta_hat = XX_inv.dot(Xy)

    n_coeff = beta_hat.size
    assert n_coeff == X.shape[1]
    
    return beta_hat

In [61]:
my_beta = ols_coeffficients(y = my_data[:,0],
                            X = my_data[:,1:], 
                            constant= True)

array([ 1.02621937, -2.06831318])

## Now Standard Errors

In [63]:
def get_residuals(dep_var, expl_var, constant = True):
    """
        YOU SHOULD WRITE DOCs
    """
    
    assert type(dep_var) == np.ndarray
    assert type(expl_var) == np.ndarray
    
    X = expl_var
    y = dep_var 
    
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert X_rows == y_rows
    n_obs = y_rows

    if constant == True:
        # add a constant to matrix
        iota     = np.ones(n_obs).reshape(n_obs, 1)
        X = np.column_stack([iota, X])
    
    ## get projection matrix
    XX_inv = np.linalg.inv(X.T.dot(X))
    P = X.dot(XX_inv).dot(X.T)
    
    ##
    P_rows, P_cols = P.shape
    assert P_rows == P_cols
    assert X_rows == P_rows
    
    M = np.eye(P_rows) - P
    
    residuals = M.dot(y)
    
    return residuals

In [66]:
resid = get_residuals(my_data[:,0], my_data[:,1:])



In [67]:
resid.shape

(10000,)

In [79]:

def compute_sigma2(dep_var, expl_var, constant = True):
    """
        WRITE DOCS
    """
    
    # type checking
    assert type(dep_var) == np.ndarray
    assert type(expl_var) == np.ndarray
    
    X = expl_var
    y = dep_var 
    
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert X_rows == y_rows
    
    # now get the residuals
    u = get_residuals(y, X, constant = constant)
    uu = u.dot(u)
    
    # now sigma2
    df = X_rows - X_cols
    sigma2 = uu / df
    
    assert np.isscalar(sigma2)
    
    return sigma2

In [80]:
compute_sigma2(my_data[:,0], my_data[:,1:])


25.501448503139596

In [81]:
def compute_vcov_homosk(dep_var, expl_var, constant = True):
    """
        WRITE DOCS
    """
    
    # type checking
    assert type(dep_var) == np.ndarray
    assert type(expl_var) == np.ndarray
    X = expl_var
    y = dep_var 
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert X_rows == y_rows
    n_obs = y_rows
    if constant == True:
        # add a constant to matrix
        iota     = np.ones(n_obs).reshape(n_obs, 1)
        X = np.column_stack([iota, X])
    
    # compute vcov matrix
    sigma2 = compute_sigma2(y,X, constant = False)
    
    XX = X.T.dot(X)
    XX_inv = np.linalg.inv(XX)
    
    vcov_mat = sigma2 * XX_inv
    
    # get std Errors
    std_errors = np.sqrt(np.diag(vcov_mat))
    
    return [std_errors, vcov_mat]

In [82]:
my_std, my_vcov = compute_vcov_homosk(my_data[:,0], my_data[:,1:])

In [83]:
my_std

array([0.13477403, 0.17436836, 0.17514583])

# HC0 standard errors?

In [90]:
def add_heterosk(u, X):
    u = u + X[:,2]**2
    return u

In [92]:
data_properties2 = { 'n_obs': 10000,
                    'beta': np.array([0.25, 0.75, -2.25]),
                    'sigma': 5,
                    'hetero': True 
                    }

In [96]:
def gen_data2(data_properties):
    """
        WRITE DOCS
    """
    import numpy
    
    n_obs = data_properties['n_obs']
    beta  = data_properties['beta']
    n_x = data_properties['beta'].shape[0] - 1
    
    # make explanatory variables
    iota     = numpy.ones(n_obs).reshape(n_obs, 1)
    expl_var = numpy.random.uniform(size=[n_obs, n_x])
    
    X = numpy.column_stack(tup = (iota, expl_var))

    # classical standard errors
    epsilon = numpy.random.normal(loc = 0, scale = data_properties['sigma'], size = n_obs)

    # add a parametrized heterosk
    if data_properties['hetero'] == True:
        print('Adding heterosk to error')
        epsilon = add_heterosk(epsilon, X)
    else:
        print('no hetero!')

    y = X @ beta + epsilon

    data = np.column_stack([y, X[:,1:]])

    return data #[y, X]

In [97]:
my_data2 = gen_data2(data_properties2)

Adding heterosk to error


In [98]:
my_beta = ols_coeffficients(y = my_data2[:,0],
                            X = my_data2[:,1:], 
                            constant= True)

In [99]:
my_beta

array([ 0.16588686,  0.76147351, -1.34621923])

In [100]:
# with normal standard errors?
my_std, my_vcov = compute_vcov_homosk(my_data2[:,0], my_data2[:,1:])

In [101]:
my_std

array([0.13378086, 0.17446806, 0.17162186])

In [118]:
def compute_vcov_hetero(dep_var, expl_var, constant = True, type = 'HC0'):
    """
        WRITE DOCS
    """
    
    # type checking
    #assert type(dep_var) == np.ndarray
    #assert type(expl_var) == np.ndarray
    X = expl_var
    y = dep_var 
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert X_rows == y_rows
    n_obs = y_rows

    print(y.shape)
    print(X.shape)
    if constant == True:
        # add a constant to matrix
        iota     = np.ones(n_obs).reshape(n_obs, 1)
        X = np.column_stack([iota, X])
    
    print(X.shape)


    if type == 'HC0':
        # compute vcov matrix
        u = get_residuals(y, X, constant =False)
        Sigma_hat = np.diag(u**2)
        XX = X.T.dot(X)
        XX_inv = np.linalg.inv(XX)

        meat = (X.T.dot(Sigma_hat)).dot(X)
        
        vcov_mat = (XX_inv.dot(meat)).dot(XX_inv)
    else:
        print('method', type, 'not supported')
    
    # get std Errors
    std_errors = np.sqrt(np.diag(vcov_mat))
    
    return [std_errors, vcov_mat]

In [119]:
my_std, my_vcov = compute_vcov_hetero(my_data2[:,0], my_data2[:,1:])

(10000,)
(10000, 2)
(10000, 3)


In [120]:
my_std

array([0.13585259, 0.17509943, 0.17362959])

In [121]:
my_std, my_vcov = compute_vcov_homosk(my_data2[:,0], my_data2[:,1:])

In [122]:
my_std

array([0.13378086, 0.17446806, 0.17162186])

# Verify we did this stuff correctly by an pre-canned OLS estimate

In [84]:
import statsmodels.api as sm

In [123]:
Y = my_data2[:,0]
X = my_data2[:,1:]
X = sm.add_constant(X)

In [124]:
model = sm.OLS(Y,X)

In [128]:
results = model.fit(cov_type='HC0')

In [129]:
results.params

array([ 0.16588686,  0.76147351, -1.34621923])

In [130]:
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     40.06
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           4.68e-18
Time:                        18:13:50   Log-Likelihood:                -30276.
No. Observations:               10000   AIC:                         6.056e+04
Df Residuals:                    9997   BIC:                         6.058e+04
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1659      0.136      1.221      0.2

In [104]:
resid**2

array([  1.73781367, 150.32162514,   6.83606561, ..., 145.70651338,
         7.8136511 ,   2.87210633])

In [105]:
np.diag(resid**2)

array([[  1.73781367,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        , 150.32162514,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   6.83606561, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., 145.70651338,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          7.8136511 ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   2.87210633]])